# Running HF Models through API and Locally in Langchain

Here we (1) test the inference API from huggingface to call a model output running in the HF server and (2) learn how we can import that model and run it locally on our macchines. All compatible with Langchanin tools

Inspiration taken from: https://www.youtube.com/watch?v=Kn7SX2Mx_Jk&list=PL8motc6AQftk1Bs42EW45kwYbyJ4jOdiZ&index=6

### ReadMe material

- [Inference in Large Language Models](https://medium.com/@andrew_johnson_4/understanding-inference-in-large-language-models-f4a4a4a736a5) - Insightful definition of text inference for LLMs (no need to pay for entire article, definition in first paragraphs)